In [82]:
import pandas as pd 
import numpy as np 
import spacy 

In [4]:
file_path = "/Users/flint/Data/recipe/italian/gz_recipe.csv"
R = pd.read_csv(filepath_or_buffer=file_path, index_col=0)

In [6]:
R.head()

,Nome,Categoria,Link,Persone/Pezzi,Ingredienti,Steps
0,Tiramisù,Dolci,https://ricette.giallozafferano.it/Tiramisu.html,8,"[['Mascarpone', '750g'], ['Uova', '260g'], ['S...",Per preparare il tiramisù preparate il caffé c...
1,Cookies,Dolci,https://ricette.giallozafferano.it/Cookies.html,12,"[['Farina 00', '195g'], ['Burro', '100g'], ['B...","Per preparare i cookies, assicuratevi che il b..."
2,Pancake allo sciroppo d'acero,Dolci,https://ricette.giallozafferano.it/Pancakes-al...,4,"[['Burro', '25g'], ['Farina 00', '125g'], ['Uo...",Iniziamo la preparazione dei pancake fondendo ...
3,Crema al mascarpone,Dolci,https://ricette.giallozafferano.it/Crema-al-ma...,4,"[['Mascarpone', '500g'], ['Zucchero', '125g'],...",Per preparare la crema al mascarpone versate i...
4,Crepe dolci e salate (ricetta base),Dolci,https://ricette.giallozafferano.it/Crepes-dolc...,15,"[['Uova', '3'], ['Farina 00', '250g'], ['Latte...",Per preparare le crepe dolci e salate iniziate...


In [20]:
ingredient_collection = []
for i in R['Ingredienti'].values:
    first = i.split('], [')
    ingredients = []
    for word in first:
        ingredients.append(word.split('", "')[0].split("', '")[0].lstrip("[").lstrip("'"))
    ingredient_collection.append(ingredients)

## Creazione del corpus
[Italian Food Recipes](https://www.kaggle.com/datasets/edoardoscarpaci/italian-food-recipes)

In [26]:
corpus = []
categories = R['Categoria']
titles = R['Nome']
steps = R['Steps']
for i, title in enumerate(titles):
    s = steps[i]
    if pd.isnull(s):
        s = ""
    ingr = ", ".join(ingredient_collection[i])
    text = "\n".join([title, ingr, s])
    corpus.append(text)

In [28]:
print(corpus[0])

Tiramisù
Mascarpone, Uova, Savoiardi, Zucchero, Caffè, Cacao amaro in polvere
Per preparare il tiramisù preparate il caffé con la moka per ottenerne 300 g, poi zuccherate a piacere (noi abbiamo messo un cucchiaino) e lasciatelo raffreddare in una ciotolina bassa e ampia. Separate le uova dividendo gli albumi dai tuorli ricordando che per montare bene gli albumi non dovranno presentare alcuna traccia di tuorlo. Montate i tuorli con le fruste elettriche, versando solo metà dose di zucchero Non appena il composto sarà diventato chiaro e spumoso, e con le fruste ancora in funzione, potrete aggiungere il mascarpone, poco alla volta Incorporato tutto il formaggio avrete ottenuto una crema densa e compatta Quando saranno schiumosi versate il restante zucchero un po’ alla volta Dovrete montarli a neve ben ferma così stempererete il composto. Dopodiché procedete ad aggiungere la restante parte di albumi, poco alla volta mescolando molto delicatamente dal basso verso l'alto La crema al mascarpon

## Creazione matrice termine documento

In [64]:
from nltk import word_tokenize

### Lemmatizzazione

In [86]:
from tqdm.notebook import tqdm 

In [84]:
nlp = spacy.load("it_core_news_lg")

In [85]:
def lemmatize(text):
    doc = nlp(text)
    return [token.lemma_.lower() for token in doc]

In [87]:
C = {}
run = list(enumerate(corpus))
for i, text in tqdm(run):
    C[i] = {}
#    for word in word_tokenize(text.replace('.', ' '), language='italian'):
    for word in lemmatize(text):
        try:
            C[i][word.lower()] += 1
        except KeyError:
            C[i][word.lower()] = 1
df = pd.DataFrame(C)

  0%|          | 0/5939 [00:00<?, ?it/s]

In [92]:
df.to_csv("/Users/flint/Data/recipe/italian/dataframe.csv")

In [88]:
df.shape

(15965, 5939)

In [89]:
df.fillna(0, inplace=True)
df 

,0,1,2,3,4,5,6,7,8,9,...,5929,5930,5931,5932,5933,5934,5935,5936,5937,5938
tiramisù,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
\n,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
mascarpone,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
",",14.0,9.0,14.0,7.0,10.0,39.0,5.0,14.0,16.0,7.0,...,11.0,12.0,12.0,7.0,24.0,15.0,18.0,13.0,13.0,20.0
uova,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
daiquiri,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dela,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
bicchieri,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
raffreddira,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [95]:
print(titles[0])
print(df[0].sort_values(ascending = False).head(5))

print(titles[10])
print(df[10].sort_values(ascending = False).head(5))

Tiramisù
il     24.0
,      14.0
uno    11.0
di     10.0
e      10.0
Name: 0, dtype: float64
New York Cheesecake
il     33.0
,      23.0
di     13.0
uno    11.0
e      10.0
Name: 10, dtype: float64


## Similarità

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
sigma = cosine_similarity(df[0].values.reshape(-1, 1).T, df.T)

In [60]:
pd.DataFrame(sigma[0], index=titles).sort_values(0, ascending=False).head(20)

,0
Nome,
Tiramisù,1.000000
Tiramisù goloso,0.783619
Cheesecake tiramisù,0.756184
Tiramisù con biscotti secchi,0.754670
Torta Saint Honoré con pan di Spagna,0.754257
Tiramisù di ricotta,0.748756
Torta mimosa classica,0.748253
Tiramisù di pandoro,0.745520
Torta Saint Honoré,0.743932
